In [37]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [42]:
torch.manual_seed(777)
idx2char = ['h', 'i', 'e', 'l', 'o']

# hihell
x_data = [0, 1, 0, 2, 3, 3]
y_data = [1, 0, 2, 3, 3, 4]

one_hot_lookup = [[1, 0, 0, 0, 0],  # 0
                  [0, 1, 0, 0, 0],  # 1
                  [0, 0, 1, 0, 0],  # 2
                  [0, 0, 0, 1, 0],  # 3
                  [0, 0, 0, 0, 1]]  # 4

x_one_hot = [one_hot_lookup[x] for x in x_data]
x_one_hot

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0]]

In [52]:
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

# HyperParameters
num_classes = 5
input_size  = 5
hidden_size = 5
batch_size = 1
sequence_length = 1
num_layers = 1

print("input:", inputs.shape)
print("label:", labels.shape)

input: torch.Size([6, 5])
label: torch.Size([6])


In [76]:
class Module(nn.Module):
    def __init__(self):
        super(Module, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
    
    def forward(self, hidden, x):
        x = x.view(batch_size, sequence_length, input_size)
        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1, num_classes)
    
    def init_hidden(self):
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))
    
model = Module()
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()
    
    print("predicted string:")
    for _input, _label in zip(inputs, labels):
        hidden, output = model(hidden, _input)
        print("output.shape:", output.shape)
        print("hidden.shape:", hidden.shape)
        val, idx = output.max(1)
        print(idx2char[idx.data[0]])
        print(_label.shape)
        
        loss += criterion(output, _label)
        
    print(", epoch: %d, loss: %1.3f" % (epoch + 1, loss.data[0]))

    loss.backward()
    optimizer.step()

Module(
  (rnn): RNN(5, 5, batch_first=True)
)
predicted string:
output.shape: torch.Size([1, 5])
hidden.shape: torch.Size([1, 1, 5])
i
torch.Size([])


IndexError: dimension specified as 0 but tensor has no dimensions